In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
import math
from prettytable import PrettyTable
from scipy import sparse
import csv
import graphviz

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, auc, make_scorer, silhouette_score
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_curve
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from wordcloud import WordCloud
from sklearn.cluster import KMeans, AgglomerativeClustering,DBSCAN

import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [2]:
%%time
data_train = pd.read_csv('./Matrices/sample_data_train.csv')
data_test = pd.read_csv('./Matrices/sample_data_test.csv')
data = pd.concat([data_train, data_test], ignore_index=True)

del data_train, data_test

Wall time: 270 ms


In [67]:
def run_dbscan(data, vects):
    sc = StandardScaler()
    
    data_np = data.drop(['Id', 'ProductId', 'UserId', 'ProfileName', 'Score', 'Summary', 'Text', 'CleanedText'], axis=1)
    data_np = np.hstack((data_np, vects))
    data_np = sc.fit_transform(data_np)
    
    scores = []; radii = [1, 10, 20, 30, 40];
    for i in radii:
        print(f'Starting Radius #{i}')
        model = DBSCAN(eps=i, min_samples=200)
        model.fit(data_np)
        print(set(model.labels_))
    
    return data_np, model

In [55]:
%%time
def avg_w2v(data_train):
    train_list_of_sent=[]
    for sent in data_train['CleanedText'].values:
        train_list_of_sent.append(sent.split())

    w2v_model = Word2Vec(train_list_of_sent, min_count=5, size=100, workers=4)
    w2v_words = list(w2v_model.wv.vocab)

    sent_vectors_train = []
    for sent in train_list_of_sent:
        sent_vec = np.zeros(100)
        count_words = 0
        for word in sent:
            if word in w2v_words:
                try:
                    vec = w2v_model.wv[word]
                except KeyError:
                    vec = np.zeros(100)
                sent_vec += vec
                count_words += 1
        if count_words != 0:
            sent_vec /= count_words
        sent_vectors_train.append(sent_vec)
    
    return sent_vectors_train

vectors = avg_w2v(data)

Wall time: 22.8 s


In [68]:
%%time
data_np, model = run_dbscan(data, vectors)

Starting Radius #1
{-1}
Starting Radius #10
{0, -1}
Starting Radius #20
{0, -1}
Starting Radius #30
{0, -1}
Starting Radius #40
{0}
Wall time: 10min 7s
